# Import Mapping

In [17]:
import ipywidgets as widgets
mappingBox = widgets.Text(value="mapping.ttl", description="URL of mapping file location:", style={"description_width": "initial"})
rdfEndpointBox = widgets.Text(value="http://graphdb:7200/repositories/r2rml", description="URL of RDF endpoint:", style={"description_width": "initial"})
widgets.VBox([mappingBox, rdfEndpointBox])

VkJveChjaGlsZHJlbj0oVGV4dCh2YWx1ZT11J21hcHBpbmcudHRsJywgZGVzY3JpcHRpb249dSdVUkwgb2YgbWFwcGluZyBmaWxlIGxvY2F0aW9uOicsIHN0eWxlPURlc2NyaXB0aW9uU3R5bGXigKY=


## Insert mapping
Execute the code below to insert the default mapping

In [18]:
rdfEndpointBox.value

u'http://graphdb:7200/repositories/r2rml'

In [19]:
mappingBox.value

u'mapping.ttl'

In [20]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

# drop all mappings
endpoint = SPARQLWrapper(rdfEndpointBox.value + "/statements")
endpoint.setQuery("DROP ALL;")
endpoint.method = "POST"
endpoint.query()

# Read RDF mapping file
from rdflib import Graph, URIRef, Literal
g = Graph()
g.parse(mappingBox.value, format="n3")
rdfString = g.serialize(format='nt')

# Create insert query
insertQuery = "INSERT { %s } WHERE { }" % rdfString

# Execute insert query
endpoint = SPARQLWrapper(rdfEndpointBox.value + "/statements")
endpoint.setQuery(insertQuery)
endpoint.method = "POST"
try:
    endpoint.query()
    print("Mapping succesfully uploaded")
except:
    print("Something went wrong during upload.")

Mapping succesfully uploaded


In [1]:
import os
from SPARQLWrapper import SPARQLWrapper
from rdflib import Graph

INFO:rdflib:RDFLib Version: 4.2.1


In [2]:
class ManageR2RML:
    """ Management of RDF mappings
    """

    def __init__(self, graphdb_base_url, r2rml_endpoint, mapping_file):
        """ Initialisation method for ManageR2RML class

        Parameters
        ----------
        graphdb_base_url : str
            GraphDB endpoint
        r2rml_endpoint : str
            R2RML endpoint
        mapping_file : str
            Path to mapping file
        """
        self.graphdb_base_url = graphdb_base_url
        self.r2rml_endpoint = r2rml_endpoint
        self.mapping_file = mapping_file

    def drop_rdf_mappings(self):
        """ Drop all RDF mappings
        """
        try:
            url = os.path.join(self.graphdb_base_url, self.r2rml_endpoint)
            endpoint = SPARQLWrapper(url)
            endpoint.setQuery('DROP ALL;')
            endpoint.method = 'POST'
            endpoint.query()
        except Exception as e:
            raise Exception('Unable to drop RDF mappings: {e}')

    def load_rdf_mapping(self):
        """ Load RDF mapping file
        """
        try:
            g = Graph()
            g.parse(self.mapping_file, format='n3')
            self.rdf_string = g.serialize(format='nt')
        except Exception as e:
            raise Exception('Unable to load RDF mapping: {e}')

    def insert_rdf_mapping(self):
        """ Insert RDF mapping to R2RML endpoint
        """
        try:
            url = os.path.join(self.graphdb_base_url, self.r2rml_endpoint)
            endpoint = SPARQLWrapper(url)
            insert_query = 'INSERT { %s } WHERE { }' % self.rdf_string
            endpoint.setQuery(insert_query)
            endpoint.method = 'POST'
            endpoint.query()
        except Exception as e:
            raise Exception('Unable to insert RDF mapping: {e}')

In [7]:
graphdb_base_url = 'http://graphdb:7200/repositories/'
r2rml_endpoint = os.path.join('r2rml', 'statements')
mapping_file = 'mapping_coral.ttl'

In [8]:
r2rml = ManageR2RML(
    graphdb_base_url=graphdb_base_url, r2rml_endpoint=r2rml_endpoint,
    mapping_file=mapping_file
)
r2rml.drop_rdf_mappings()

In [9]:
r2rml.load_rdf_mapping()

In [10]:
r2rml.insert_rdf_mapping()